In [5]:
import pandas as pd
import numpy as np 
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup

In [6]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text
# Parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# Create a list to store neighbourhood data
neighborhoodList = []
# Append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
  neighborhoodList.append(row.text)
# Create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,A. C. Guards
1,A. S. Rao Nagar
2,Abhyudaya Nagar
3,Abids
4,Adibatla


In [12]:
pip install geocoder

     |████████████████████████████████| 98 kB 7.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
import geocoder

# Defining a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist()]
coords

[[17.395014876941943, 78.45981230832962],
 [17.411200000000065, 78.50824000000006],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.235790000000065, 78.54130000000004],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.389149685816093, 78.46385388458206],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.53332000000006, 78.32529000000005],
 [17.435350000000028, 78.44861000000003],
 [17.45787000000007, 78.53882000000004],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.388960000000054, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],


In [15]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
print(kl_df.shape)
kl_df
(200,3)


(200, 3)


(200, 3)

In [16]:
kl_df

,Neighborhood,Latitude,Longitude
0,A. C. Guards,17.395015,78.459812
1,A. S. Rao Nagar,17.411200,78.508240
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adibatla,17.235790,78.541300
...,...,...,...
195,Serilingampally,17.482160,78.323000
196,Shah-Ali-Banda,17.357390,78.473200
197,Shahran Market,17.364890,78.476290
198,Shanker Mutt,17.399817,78.507919


In [18]:
# Getting the coordinates of Hyderabad
address = 'Hyderabad, India'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.46106473453146.


In [20]:
pip install folium

     |████████████████████████████████| 93 kB 2.9 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
import folium
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)
# Adding markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'],  kl_df['Longitude'], kl_df['Neighborhood']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_kl)
map_kl

In [24]:
CLIENT_ID = 'NS2I2NM31E1ZP2VAB5C4WX5WZMC51PPRVLMQALYARK3USYA1' # your Foursquare ID
CLIENT_SECRET = 'XIVKBDDOSCBSECVR2AQ450TRAXKMFPGA41OVAXJMUCZUCWZO' # your Foursquare Secret
VERSION = '20201112'
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
# Create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
# Make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
# Return only relevant information for each nearby venue
for venue in results:
    venues.append((neighborhood,lat,long,venue['venue']['name'],
    venue['venue']['location']['lat'],venue['venue']['location']    ['lng'],venue['venue']['categories'][0]['name']))

In [25]:
venues_df = pd.DataFrame(venues)
# Defining the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(24, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Shivam Road,17.392602,78.510801,Subway,17.404173,78.514950,Sandwich Place
1,Shivam Road,17.392602,78.510801,Domino's Pizza,17.392508,78.510827,Pizza Place
2,Shivam Road,17.392602,78.510801,Raghavendra Tiffins,17.399157,78.512198,Indian Restaurant
3,Shivam Road,17.392602,78.510801,Baskin-Robbins,17.404311,78.510034,Ice Cream Shop
4,Shivam Road,17.392602,78.510801,Cafe Coffee Day,17.393138,78.497436,Coffee Shop


In [27]:
# Lets check how many venues were returned for each neighbourhood
venues_df.groupby(["Neighborhood"]).count()
# Lets check out how many unique categories can be curated from all the returned values
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))
#There are 174 unique categories
# Displaying the first 50 Venue Category names
venues_df['VenueCategory'].unique()[:50]


There are 17 unique categories.


array(['Sandwich Place', 'Pizza Place', 'Indian Restaurant',
       'Ice Cream Shop', 'Coffee Shop', 'Café', 'Food', 'Train Station',
       'Chinese Restaurant', 'Platform', 'Electronics Store',
       'Convenience Store', 'Motel', 'Clothing Store', 'Gym',
       'Food & Drink Shop', 'Diner'], dtype=object)

In [29]:
# One hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]
print(kl_onehot.shape)


(24, 18)


In [30]:
kl_grouped=kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
print(kl_grouped.shape)
kl_grouped

(1, 18)


,Neighborhoods,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Diner,Electronics Store,Food,Food & Drink Shop,Gym,Ice Cream Shop,Indian Restaurant,Motel,Pizza Place,Platform,Sandwich Place,Train Station
0,Shivam Road,3,1,1,3,1,1,1,1,1,2,2,2,1,1,1,1,1


In [36]:
len((kl_grouped[kl_grouped["Chinese Restaurant"] > 0]))

1

In [38]:
kl_mall = kl_grouped[["Neighborhoods","Chinese Restaurant"]]

In [43]:
# Setting the number of clusters
from sklearn.cluster import KMeans
kclusters = 1
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)
# Run k-means clustering algorithm
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(kl_clustering)
# Checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0], dtype=int32)

In [44]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()
# Add the clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

,Neighborhood,Chinese Restaurant,Cluster Labels
0,Shivam Road,1,0


In [45]:
kl_merged['Latitude'] = kl_df['Latitude']
kl_merged['Longitude'] = kl_df['Longitude']
# Sorting the results by Cluster Labels
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,Shivam Road,1,0,17.395015,78.459812


In [50]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# Creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters
